In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

#data_folder = Path('/sgoinfre/goinfre/Perso/pdespres/data/')
data_folder = Path('E:/Downloads/')
#data_folder = Path('/sgoinfre/goinfre/Perso/pdespres/data/')

df = pd.read_csv(data_folder/'test_redux.csv', \
                 dtype='int8')
y = pd.read_csv(data_folder/'y2.csv', \
                sep=';', usecols=[1], dtype='int8')

print(df.shape, y.shape)

(13250, 18124) (26495, 1)


In [2]:
snp_list = ['SNP_6292', 'SNP_8285', 'SNP_10188', 'SNP_11870', 'SNP_12565', 'SNP_13326']

def f(row):
    val = 0
    if row['SNP_8927'] == i and row[snp] == j:
        val = 2
    return val

sup_df = df[snp_list].copy()
sup_df['SNP_8927'] = df.SNP_8927
for snp in snp_list:
    for i in range (0,3):
        for j in range (0,3):
            sup_df['S8927-' + snp + '-' + (str(i) + str(j))] = sup_df.apply(f, axis=1)
sup_df.drop(snp_list, axis=1, inplace=True)
sup_df.drop('SNP_8927', axis=1, inplace=True)
sup_df.shape

(13250, 54)

In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgb
from scipy.optimize import minimize
from sklearn.externals import joblib
from keras.models import load_model
from itertools import chain

# cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
# for train_index, test_index in cv.split(df, y):
#     train_x, train_y = df.iloc[train_index], y.iloc[train_index].values.ravel()
#     test_x, test_y = df.iloc[test_index], y.iloc[test_index].values.ravel()
    
test_x = df

predictions = []
starting_values = []

rfc = joblib.load(data_folder/'RF_7862.pkl')
lrc = joblib.load(data_folder/'LR_sup_8389.pkl') 
svc = joblib.load(data_folder/'SVC_9925.pkl') 
xgb = joblib.load(data_folder/'XGB_sup_8638.pkl') 
mlp = load_model(data_folder/'MLP_8468.pkl') 

t1 = pd.read_csv(data_folder/'irf_weight_00010.csv', names=['a','b'])
t1.drop(['a'], axis=1, inplace=True)
t1 = t1.drop(t1.index[:1])
rfc_test_x = test_x.drop(t1['b'].tolist(), axis=1)
probas_ = rfc.predict_proba(rfc_test_x)[:, 1]
predictions.append(probas_)
# score = roc_auc_score(test_y, probas_)
# starting_values.append(score)
# print('RFC AUC {score}'.format(score=score))

rfc_test_x = pd.concat([rfc_test_x, sup_df], axis=1)
probas_ = lrc.predict_proba(rfc_test_x)[:, 1]
predictions.append(probas_)
# score = roc_auc_score(test_y, probas_)
# starting_values.append(score)
# print('LRC AUC {score}'.format(score=score))

t1 = pd.read_csv(data_folder/'drop_corr90.csv', names=['a','b'])
t1.drop(['b'], axis=1, inplace=True)
t1 = t1.drop(t1.index[:1])
svc_test_x = test_x.drop(t1['a'].tolist(), axis=1)
probas_ = svc.predict_proba(svc_test_x)[:, 1]
predictions.append(probas_)
# score = roc_auc_score(test_y, probas_)
# starting_values.append(score)
# print('SVC AUC {score}'.format(score=score))

t1 = pd.read_csv(data_folder/'drop_corr98.csv', names=['a','b'])
t1.drop(['b'], axis=1, inplace=True)
t1 = t1.drop(t1.index[:1])
t2 = pd.read_csv(data_folder/'irf_weight_00000.csv', names=['a','b'])
t2.drop(['a'], axis=1, inplace=True)
t2.columns = ['a']
t2 = t2.drop(t2.index[:1])
concatenated = pd.concat([t1, t2], axis=0)
concatenated.drop_duplicates('a')
mlp_test = test_x.drop(concatenated['a'].tolist(), axis=1)
probas_ = mlp.predict_proba(mlp_test)
predictions.append(list(chain.from_iterable(probas_)))
retest = probas_
# score = roc_auc_score(test_y, probas_)
# starting_values.append(score)
# print('MLP AUC {score}'.format(score=score))

test_x = pd.concat([test_x, sup_df], axis=1)
probas_ = xgb.predict_proba(test_x)[:, 1]
predictions.append(probas_)
# score = roc_auc_score(test_y, probas_)
# starting_values.append(score)
# print('XGB AUC {score}'.format(score=score))

Using TensorFlow backend.


In [5]:
from scipy.stats import mstats

test = np.array(predictions)
#probas_f = np.mean(test.T, axis=1)
probas_f = mstats.gmean(test.T, axis=1)
probas_f
# score = roc_auc_score(test_y, test)
# print(np.mean(starting_values))
# print('Ensemble AUC {score}'.format(score=score) + '\n')
#array([0.11863515, 0.08671564, 0.86513838, ..., 0.31882419, 0.14731153, 0.60631298])

array([0.02304813, 0.03376106, 0.83567965, ..., 0.23033955, 0.0422533 ,
       0.57712089])

In [6]:
submission = pd.DataFrame(columns={'Ids', 'TARGET'})
submission['Ids'] = [i for i in range(26500,39750)]
submission['TARGET'] = probas_f
submission = submission[['Ids', 'TARGET']]
submission.to_csv(data_folder/'submission_gmean.csv', sep=';', index=False, header={'Ids', 'TARGET'})